# Altair and Panel: Declarative and Dashboard Visualization

Demonstrates declarative charts with Altair and composable dashboards with Panel.

**Libraries:**
- [Altair](https://altair-viz.github.io/) — Declarative visualization based on Vega-Lite grammar
- [Panel](https://panel.holoviz.org/) — High-level app and dashboard framework
- [hvplot](https://hvplot.holoviz.org/) — High-level plotting API for pandas/xarray

> All outputs are **interactive HTML** — charts support zoom, pan, hover tooltips, and cross-filtering.

In [ ]:
import numpy as np
import pandas as pd
import altair as alt
import panel as pn
import hvplot.pandas

pn.extension('vega')  # enable Vega/Altair support in Panel
alt.renderers.enable('default')

In [ ]:
rng = np.random.default_rng(seed=42)

# Simulated stock prices
dates = pd.date_range('2023-01-01', periods=252, freq='B')
companies = ['TechCorp', 'EnergyBiz', 'HealthInc', 'RetailCo']
prices = pd.DataFrame(
    {c: 100 * np.cumprod(1 + rng.normal(0.0003, 0.015, len(dates))) for c in companies},
    index=dates,
)
prices.index.name = 'date'
prices_long = prices.reset_index().melt(id_vars='date', var_name='company', value_name='price')

# Scatter dataset
n = 80
scatter_df = pd.DataFrame({
    'revenue_growth': rng.normal(0.12, 0.08, n),
    'profit_margin': rng.normal(0.15, 0.07, n),
    'market_cap_B': rng.lognormal(3, 1.2, n),
    'sector': rng.choice(['Tech', 'Finance', 'Health', 'Energy', 'Consumer'], n),
    'ticker': [f'S{i:03d}' for i in range(n)],
})

print(f"Price data: {prices_long.shape} rows")
print(f"Scatter data: {scatter_df.shape} rows")
prices.tail(3)

## Altair: Line Chart with Brush Navigator

**Interaction:** Click and drag on the bottom overview chart to zoom the top detail chart.

In [ ]:
brush = alt.selection_interval(encodings=['x'])

detail = (
    alt.Chart(prices_long, title="Stock Prices — Drag lower chart to zoom")
    .mark_line(strokeWidth=1.8)
    .encode(
        x=alt.X('date:T', title='Date'),
        y=alt.Y('price:Q', title='Price (USD)', scale=alt.Scale(zero=False)),
        color=alt.Color('company:N', legend=alt.Legend(title='Company')),
        tooltip=['date:T', 'company:N', alt.Tooltip('price:Q', format='.2f')],
    )
    .properties(width=700, height=280)
    .transform_filter(brush)
)

overview = (
    alt.Chart(prices_long)
    .mark_line(strokeWidth=1.2, opacity=0.7)
    .encode(
        x=alt.X('date:T', title=''),
        y=alt.Y('price:Q', title='Price', axis=alt.Axis(labels=False)),
        color=alt.Color('company:N', legend=None),
    )
    .properties(width=700, height=80)
    .add_params(brush)
)

detail & overview

## Altair: Linked Scatter + Histogram

**Interaction:** Drag on the scatter plot to filter the histogram. Click legend items to highlight sectors.

In [ ]:
color_scale = alt.Scale(
    domain=['Tech', 'Finance', 'Health', 'Energy', 'Consumer'],
    range=['#4C72B0', '#DD8452', '#55A868', '#C44E52', '#8172B2'],
)

point_select = alt.selection_point(fields=['sector'], bind='legend')
brush_scatter = alt.selection_interval()

scatter = (
    alt.Chart(scatter_df, title="Profit Margin vs Revenue Growth")
    .mark_circle(opacity=0.75, stroke='white', strokeWidth=0.5)
    .encode(
        x=alt.X('revenue_growth:Q', title='Revenue Growth', axis=alt.Axis(format='%')),
        y=alt.Y('profit_margin:Q', title='Profit Margin', axis=alt.Axis(format='%')),
        size=alt.Size('market_cap_B:Q', scale=alt.Scale(range=[40, 600])),
        color=alt.Color('sector:N', scale=color_scale),
        tooltip=['ticker:N', 'sector:N',
                 alt.Tooltip('revenue_growth:Q', format='.1%'),
                 alt.Tooltip('profit_margin:Q', format='.1%'),
                 alt.Tooltip('market_cap_B:Q', format='.1f', title='Mkt Cap $B')],
        opacity=alt.condition(point_select, alt.value(0.85), alt.value(0.15)),
    )
    .properties(width=420, height=320)
    .add_params(point_select, brush_scatter)
)

histogram = (
    alt.Chart(scatter_df)
    .mark_bar(opacity=0.8)
    .encode(
        x=alt.X('revenue_growth:Q', bin=alt.Bin(maxbins=20), title='Revenue Growth'),
        y=alt.Y('count()', title='Count'),
        color=alt.Color('sector:N', scale=color_scale),
    )
    .transform_filter(brush_scatter)
    .properties(width=420, height=160, title='Distribution of Selected Points')
)

scatter & histogram

## Altair: Annotated Correlation Heatmap

In [ ]:
monthly_returns = prices.resample('ME').last().pct_change().dropna()
corr = monthly_returns.corr().reset_index().melt(id_vars='index', var_name='variable', value_name='correlation')
corr.columns = ['company_x', 'company_y', 'correlation']

heatmap = (
    alt.Chart(corr, title="Monthly Return Correlations")
    .mark_rect()
    .encode(
        x=alt.X('company_x:N', title=''),
        y=alt.Y('company_y:N', title=''),
        color=alt.Color('correlation:Q',
                        scale=alt.Scale(scheme='redblue', domain=[-1, 1]),
                        legend=alt.Legend(title='Correlation')),
        tooltip=['company_x:N', 'company_y:N', alt.Tooltip('correlation:Q', format='.3f')],
    )
    .properties(width=320, height=300)
)

text_layer = heatmap.mark_text(fontSize=13).encode(
    text=alt.Text('correlation:Q', format='.2f'),
    color=alt.condition(alt.datum.correlation > 0.5, alt.value('white'), alt.value('black')),
)

heatmap + text_layer

## hvplot: Quick Exploratory Charts

hvplot extends pandas DataFrames with a `.hvplot` accessor for one-liner interactive charts.

In [ ]:
prices.hvplot.line(
    title="Stock Prices (hvplot)",
    ylabel="Price (USD)",
    xlabel="Date",
    width=720,
    height=340,
    legend='top_left',
)

In [ ]:
monthly_returns.hvplot.hist(
    bins=20,
    title="Monthly Return Distribution",
    ylabel="Count",
    xlabel="Monthly Return",
    width=720,
    height=300,
    alpha=0.7,
)

## Panel: Composable Dashboard

Panel wraps any visualization library into servable, composable layouts with widgets.
Run `panel serve 16_altair_panel_viz.ipynb` to launch as a live app.

In [ ]:
final_prices = prices.iloc[-1]
initial_prices = prices.iloc[0]
ytd_returns = (final_prices / initial_prices - 1) * 100

def kpi_card(title, value, delta, positive):
    color = '#27ae60' if positive else '#e74c3c'
    return pn.pane.HTML(
        f"""
        <div style="background:#f8f9fa;border-radius:8px;padding:16px 20px;
                    border-left:4px solid {color};font-family:sans-serif;">
            <div style="color:#666;font-size:12px;text-transform:uppercase;">{title}</div>
            <div style="font-size:26px;font-weight:700;color:#2c3e50;">{value}</div>
            <div style="color:{color};font-size:13px;">{delta}</div>
        </div>
        """,
        width=200, height=100,
    )

kpis = pn.Row(*[
    kpi_card(
        title=c,
        value=f"${final_prices[c]:.1f}",
        delta=f"YTD: {ytd_returns[c]:+.1f}%",
        positive=ytd_returns[c] >= 0,
    )
    for c in companies
])

stock_chart = (
    alt.Chart(prices_long)
    .mark_line(strokeWidth=1.8)
    .encode(
        x=alt.X('date:T'),
        y=alt.Y('price:Q', scale=alt.Scale(zero=False)),
        color='company:N',
        tooltip=['date:T', 'company:N', alt.Tooltip('price:Q', format='.2f')],
    )
    .properties(width=750, height=300, title="Price History")
)

dashboard = pn.Column(
    pn.pane.HTML("<h2 style='font-family:sans-serif;'>Market Dashboard</h2>"),
    pn.layout.Divider(),
    pn.pane.HTML("<h4 style='font-family:sans-serif;'>KPIs</h4>"),
    kpis,
    pn.layout.Divider(),
    pn.pane.Vega(stock_chart.to_dict(), height=350),
)

dashboard

---
## Summary

1. **Altair** — Grammar-of-graphics approach: compose charts from encodings + transforms + selections
2. **Linked selections** — brush + filter enables cross-chart interaction with no JavaScript
3. **hvplot** — One-liner exploratory charts that integrate with Panel for dashboards
4. **Panel** — Wraps any visualization into a servable, widget-driven app

**Resources:**
- [Altair Gallery](https://altair-viz.github.io/gallery/)
- [Panel Getting Started](https://panel.holoviz.org/getting_started/)
- [hvplot User Guide](https://hvplot.holoviz.org/user_guide/)